<font size = 6>**Pupil Labs Blink Detection**

In [1]:
import pathlib
import numpy as np

from blink_detector.blink_detector import blink_detection_pipeline

<font size = 5>**(1) Run blink detection pipeline using an example Neon recording**

**(1.1) Load example recording and apply preprocessing**

In [7]:
recording_path = pathlib.Path("blink_detector/data/padel_tennis_neon_03-2ded8f56")

blink_event_generator = blink_detection_pipeline(recording_path, is_neon=True)

# obtain list of blink events from generator object
blink_events = list(blink_event_generator)

In [14]:
# print number of blinks (length of list), mean blink duration and blink frequency
print(
    f"Number of blinks: {len(blink_events)}\n"
    f"Mean blink duration: {np.mean([(blink_event.end_time-blink_event.start_time)/1e9 for blink_event in blink_events])}\n"
    # f"Blink frequency: {len(blink_events) / recording_path.with_suffix('').name.split('_')[-1]}"
)

Number of blinks: 61
Mean blink duration: 0.2649380819672132



0.310377

**(1.2) Load classifier as well as default optical flow and postprocessing parameters**

In [3]:
clf = get_classifier(is_neon=True)

of_params = OfParams()
pp_params = PPParams()

grid = create_grid(of_params.img_shape, of_params.grid_size)

NameError: name 'get_classifier' is not defined

**(1.3) Predict blink events from video frames**

In [4]:
images_timestamps = zip(zip(left_images, right_images), timestamps)

x = calculate_optical_flow(images_timestamps, of_params, grid)
x = predict_class_probas(x, clf, of_params)
x = smooth_probas(x, pp_params)
x = threshold_probas(x, pp_params)
x = compile_into_events(x)
x = filter_events(x)

blink_events = list(extract_blink_events(x, pp_params))

<font size = 5><b>Visualize blinks</b><br><br>
<font size=3>In order to visualize the blinks, a white square is drawn into the center of each frame <br>
that is part of a blink event. Next, a short video is generated from the frames.<br>

In [5]:
from IPython.display import HTML
from blink_detector.helper import generate_animation

# only aniimate the first N samples, otherwise creating the animation can take long
video_length = 3000

blink_indices = np.zeros_like(timestamps)

for blink_event in blink_events:
    on = np.where(blink_event.start_time == timestamps)[0]
    off = np.where(blink_event.end_time == timestamps)[0]
    blink_indices[int(on) : int(off)] = 1

anim = generate_animation(
    left_images[:video_length, :, :],
    right_images[:video_length, :, :],
    indices=blink_indices[:video_length],
)

HTML(anim.to_html5_video())

<font size = 5><b>(2) Blink detection in real time using PL's realtime API</b>

<font size = 3>In the next steps, blink events will be detected in (quasi) real-time using Pupil Lab's Real Time API. <br>
Note that the temporal resolution of the blink detection is restricted by the optical flow parameters<br>
(i.e., the number of layers and the layer interval), as well as postprocessing parameters (e.g. the minimum blink duration).

**(2.1) Load all relevant modules and functions**

In [1]:
from blink_detector.helper import video_stream
from blink_detector.blink_detector import blink_detection_pipeline
from pupil_labs.realtime_api.simple import discover_one_device
import nest_asyncio
from itertools import tee

**(2.2) Set up real time API**

In [2]:
import time

# needed when running in notebook
nest_asyncio.apply()

# calling the functions shortly after one another can cause an error
time.sleep(1)

device = discover_one_device()

Printing the device name and its IP helps to make sure that the discovered device is indeed <br>
the device you would like to connect with.

In [3]:
print(f"Phone IP address: {device.phone_ip}")
print(f"Phone name: {device.phone_name}")

Phone IP address: 192.168.20.132
Phone name: Kai's Companion


**(2.3) Set up real-time data stream**

In [4]:

# create three independent generator objects from the video stream
stream_left, stream_right, stream_ts = tee(video_stream(device), 3)

left_images = (left for left, _, _ in stream_left)
right_images = (right for _, right, _ in stream_right)

# timestamps need to be converted to ns
timestamps = (1e9 * timestamp for _, _, timestamp in stream_ts)

**(2.4) Run blink detection pipeline**

Get classifier path

In [5]:
import pathlib
# from blink_detector.helper import get_clf_path
# clf_path = get_clf_path(is_neon=True)
clf_path = pathlib.Path("/Users/tpfeffer/projects/real-time-blink-detection/blink_detector/weights/xgb_neon_171.sav")

Detect and print blink events in (quasi) real-time

In [ ]:
for blink_events in blink_detection_pipeline(
    left_images, 
    right_images, 
    timestamps, 
    clf_path
):
    print(blink_events)